## Initial Load of M.csv

In [36]:
import csv
def read_in_matrix(file_name):
    # initialize dictionary
    adjacency_dict = {}
    # read in file
    with open(file_name, newline='') as f:
        reader = csv.reader(f)
        # for each row
        for row in reader:
            # get the three values
            input_link = row[0]
            output_link = row[1]
            # if the input link is not in the dict add it
            if input_link not in adjacency_dict.keys():
                adjacency_dict[input_link] = [output_link]
            # otherwise just append the output link to its list
            else:
                adjacency_dict[input_link].append(output_link)
            # if the output link is not in the dict, add it with empty list
            if output_link not in adjacency_dict.keys():
                adjacency_dict[output_link] = []
    # get initial ranks, return the new dict containing all information
    return {in_link: (1.0/len(adjacency_dict), out_links) for in_link, out_links in adjacency_dict.items()}
            
dictionary = read_in_matrix('M.csv')
print(dictionary)

{'1': (0.25, ['2', '3', '4']), '2': (0.25, ['1', '4']), '3': (0.25, ['1']), '4': (0.25, ['2', '3'])}


## Map Initial Load

In [41]:
def mapper(dictionary, link):
    p = dictionary[str(link)][0]/len(dictionary[str(link)][1])
    return {out_link: (p, dictionary[out_link][1]) for out_link in dictionary[str(link)][1]}

map_results = []
for link in dictionary.keys():
    map_results.append(mapper(dictionary, link))

### Print Results

In [42]:
print(map_results)

[{'1': (0.10416666666666666, ['2', '3', '4']), '4': (0.10416666666666666, ['2', '3'])}, {'1': (0.20833333333333331, ['2', '3', '4'])}, {'2': (0.10416666666666666, ['1', '4']), '3': (0.10416666666666666, ['1'])}, {'2': (0.125, ['1', '4']), '3': (0.125, ['1']), '4': (0.125, ['2', '3'])}]


## Reduce Mapped Results

In [43]:
def reducer(map_results):
    new_ranks = {}
    for result in map_results:
        for key, value in result.items():
            if key not in new_ranks:
                new_ranks[key] = value
            else:
                new_ranks[key] = (value[0] + new_ranks[key][0], new_ranks[key][1])
    return new_ranks

dictionary = reducer(map_results)

### Print Results

In [44]:
print(dictionary)

{'1': (0.3125, ['2', '3', '4']), '4': (0.22916666666666666, ['2', '3']), '2': (0.22916666666666666, ['1', '4']), '3': (0.22916666666666666, ['1'])}
